# 第5章 モデルの評価
## 5.2 バリデーションの手法
### 5.2.1 hold-out法

In [2]:
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

cancer=load_breast_cancer()
X=cancer.data
y=cancer.target

In [12]:
tr_x,va_x,tr_y,va_y=train_test_split(X,y,test_size=0.25,random_state=False,shuffle=True)
tr_x

array([[1.185e+01, 1.746e+01, 7.554e+01, ..., 9.140e-02, 3.101e-01,
        7.007e-02],
       [1.122e+01, 1.986e+01, 7.194e+01, ..., 2.022e-02, 3.292e-01,
        6.522e-02],
       [2.013e+01, 2.825e+01, 1.312e+02, ..., 1.628e-01, 2.572e-01,
        6.637e-02],
       ...,
       [9.436e+00, 1.832e+01, 5.982e+01, ..., 5.052e-02, 2.454e-01,
        8.136e-02],
       [9.720e+00, 1.822e+01, 6.073e+01, ..., 0.000e+00, 1.909e-01,
        6.559e-02],
       [1.151e+01, 2.393e+01, 7.452e+01, ..., 9.653e-02, 2.112e-01,
        8.732e-02]])

In [14]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()
model.fit(tr_x,tr_y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [16]:
va_pred=model.predict(va_x)
score=log_loss(va_y,va_pred)
score

4.106025157383023

In [19]:
from sklearn.model_selection import KFold

#KFoldクラスを用いてhold-out法で分割する
kf=KFold(n_splits=4,shuffle=True,random_state=71)
tr_idx,va_idx=list(kf.split(X))[0]
tr_idx

array([  0,   1,   2,   4,   5,   7,   8,  10,  11,  12,  13,  14,  16,
        17,  19,  21,  22,  23,  25,  27,  28,  29,  30,  31,  33,  34,
        36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  47,  48,  51,
        52,  54,  55,  56,  57,  58,  60,  61,  62,  63,  65,  66,  67,
        68,  70,  71,  72,  73,  74,  75,  76,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  89,  90,  91,  92,  93,  96,  97,  99,
       100, 101, 102, 103, 106, 107, 108, 109, 111, 112, 113, 115, 117,
       118, 119, 120, 121, 123, 124, 125, 126, 128, 130, 132, 133, 134,
       135, 136, 138, 140, 141, 142, 143, 144, 145, 146, 149, 151, 153,
       155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 167, 168,
       169, 170, 171, 172, 174, 175, 176, 177, 178, 179, 180, 181, 183,
       186, 187, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199,
       200, 203, 204, 205, 206, 207, 209, 211, 213, 214, 215, 217, 218,
       219, 220, 221, 223, 225, 226, 228, 229, 230, 233, 234, 23

In [24]:
len(X[tr_idx])

426

### 5.2.2 クロスバリデーション

In [28]:
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
import numpy as np

scores=[]

kf=KFold(n_splits=4,shuffle=True,random_state=71)
for tr_idx,va_idx in kf.split(X):
    tr_x,va_x=X[tr_idx],X[va_idx]
    tr_y,va_y=y[tr_idx],y[va_idx]
    
    model=DecisionTreeClassifier()
    model.fit(tr_x,tr_y)
    va_pred=model.predict(va_x)
    score=log_loss(va_y,va_pred)
    scores.append(score)
    
print(scores)

[1.6907149256070286, 3.4052708923293027, 3.405259630393666, 1.2161709913158034]


### 5.2.3 stratified k-fold

In [35]:
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import numpy as np

from sklearn.datasets import load_iris
iris = load_iris()
X=iris.data
y=iris.target

In [45]:
kf=StratifiedKFold(n_splits=4,shuffle=True,random_state=71)
print((y[list(kf.split(X,y))[0][0]]==0).sum())
print((y[list(kf.split(X,y))[0][0]]==1).sum())
print((y[list(kf.split(X,y))[0][0]]==2).sum())

37
38
37


### 5.2.4 group k-fold

In [51]:
import pandas as pd
df=pd.DataFrame({'userID':['A1','B1','C1'],'purchase':[100,200,100]})
df

,userID,purchase
0,A1,100
1,B1,200
2,C1,100


In [53]:
df['userID'].isin(['A1','C1'])

0     True
1    False
2     True
Name: userID, dtype: bool

### 5.2.5 leave-one-out

In [56]:
from sklearn.model_selection import LeaveOneOut
loo=LeaveOneOut()
list(loo.split(X))[0]

(array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149]),
 array([0]))

## 5.3 時系列データのバリデーション手法
### 5.3.2 時系列データのクロスバリデーション

In [62]:
from sklearn.model_selection import TimeSeriesSplit

tss=TimeSeriesSplit(n_splits=4)
list(tss.split(X))[0]

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46,
        47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]))

In [63]:
list(tss.split(X))[1]

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59]),
 array([60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76,
        77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]))